In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import datetime
import json
from sqlalchemy.types import BigInteger, Time, Date, Float, Integer, Boolean, Text, TIMESTAMP

In [2]:
import sys

In [3]:
sys.path.append('python_scripts/')

In [4]:
import db_connection

In [13]:
df = pd.read_csv('python_scripts/content.csv', sep=';')

In [14]:
df.head()

,weather_id,date,time,city_name,temp_c,temp_f
0,202301-3111-1137-63b38217-d105-446c-813e-2a5b5...,2023-01-31,11:11:37.725363,Moscow,0.0,32.0
1,202301-3111-1138-335b50a3-106d-43df-b068-ead19...,2023-01-31,11:11:38.966144,London,9.0,48.0
2,202301-3111-1220-8983c99b-1dfb-4ec9-b84d-f684c...,2023-01-31,11:12:20.495912,Moscow,0.0,32.0
3,202301-3111-1221-5a424e50-eb1b-4254-9307-0b063...,2023-01-31,11:12:21.698796,London,9.0,48.0
4,202301-3114-5648-78fa2de4-0ccd-49c5-9ae2-80e79...,2023-01-31,14:56:48.598630,Moscow,0.0,32.0


In [16]:
engine = db_connection.engine

weather_dtypes = {'time': Time, 
        'date': Date, 
        'temp_c': Float, 
        'temp_f': Float}

In [17]:
weather_sql_query = '''
CREATE TABLE IF NOT EXISTS weather_prod(
weather_id TEXT NOT NULL PRIMARY KEY,
date DATE NOT NULL,
time TIME NOT NULL,
city_name TEXT NOT NULL,
temp_c FLOAT NOT NULL,
temp_f FLOAT NOT NULL);

INSERT INTO weather_prod (weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f)
(select weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f
from weather_tmp);
    '''

In [19]:
db_connection.load_df_to_tpm_table(df, 'weather',  weather_dtypes)

weather loading into DB...
weather_tmp loaded


In [20]:
db_connection.load_tmp_table_to_prod_table(weather_sql_query)

In [25]:
sql_query = f'select * from weather_prod'
df_from_db = pd.read_sql_query(sql_query, con=engine)

In [26]:
df_from_db

,weather_id,date,time,city_name,temp_c,temp_f
0,202301-3111-1137-63b38217-d105-446c-813e-2a5b5...,2023-01-31,11:11:37.725363,Moscow,0.0,32.0
1,202301-3111-1138-335b50a3-106d-43df-b068-ead19...,2023-01-31,11:11:38.966144,London,9.0,48.0
2,202301-3111-1220-8983c99b-1dfb-4ec9-b84d-f684c...,2023-01-31,11:12:20.495912,Moscow,0.0,32.0
3,202301-3111-1221-5a424e50-eb1b-4254-9307-0b063...,2023-01-31,11:12:21.698796,London,9.0,48.0
4,202301-3114-5648-78fa2de4-0ccd-49c5-9ae2-80e79...,2023-01-31,14:56:48.598630,Moscow,0.0,32.0
5,202301-3114-5650-9909fa3c-fb08-4ff7-8122-488ff...,2023-01-31,14:56:50.224627,London,11.0,52.0
6,202301-3114-5914-d214156e-f060-4be9-9ade-3732f...,2023-01-31,14:59:14.541966,Moscow,0.0,32.0
7,202301-3114-5915-1a4498b3-ba61-4e80-82c4-703c8...,2023-01-31,14:59:15.815832,London,11.0,52.0
